
# 📘 UC22 – Aula 7 — Agrupamentos com `groupby()` (Pandas)

![Python](https://img.shields.io/badge/Python-3.11+-blue?logo=python)
![Pandas](https://img.shields.io/badge/Pandas-groupby-green?logo=pandas)
![Google Colab](https://img.shields.io/badge/Google%20Colab-Notebook-yellow?logo=googlecolab)
![Tempo](https://img.shields.io/badge/Dura%C3%A7%C3%A3o-90%20min-red)
![N%C3%ADvel](https://img.shields.io/badge/N%C3%ADvel-Iniciante%E2%9E%9CIntermedi%C3%81rio-purple)

**Turma:** 3º Ano – Ensino Médio Técnico em Informática  
**Tema:** Agrupamentos e estatísticas por grupo com `groupby()`  
**Ferramenta principal:** Google Colab

> “Agrupar é comparar com contexto: só entendemos um valor quando olhamos o **grupo** ao redor.” – IAra 🧠



## 🎯 Objetivos de Aprendizagem
- Usar `groupby()` para **agrupar por uma ou mais colunas**.  
- Calcular **estatísticas** por grupo: `count`, `mean`, `sum`, `min`, `max`, `median`, `std`.  
- Aplicar **agregações múltiplas** com `.agg({...})`.  
- **Ordenar** e **formatar** resultados agregados.  
- Criar **Top-N por grupo** com `head`, `nlargest` e `sort_values`.  
- (Opcional) **Plotar** gráficos simples a partir de agregações.



## 📂 Preparação do Ambiente (Google Drive)
Usaremos os mesmos arquivos das aulas anteriores em **`Meu Drive/UC22/`**:

- `pokemons_pokedex.csv`  
- `escolas_estaduais_censo_escolar_2023.csv`


In [ ]:

# 📂 Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 🐍 Importações essenciais
import pandas as pd

# 🗂️ Definição dos caminhos (ajuste conforme sua pasta)
caminho_poke = "/content/drive/MyDrive/UC22/pokemons_pokedex.csv"
caminho_escolas = "/content/drive/MyDrive/UC22/escolas_estaduais_censo_escolar_2023.csv"

# 📥 Leitura dos datasets
df_poke = pd.read_csv(caminho_poke, encoding="utf-8")
df_escolas = pd.read_csv(caminho_escolas, encoding="latin-1", sep=",")

# 🧹 Padronização de nomes de colunas
df_poke.columns    = df_poke.columns.str.strip().str.lower().str.replace(" ", "_", regex=False)
df_escolas.columns = df_escolas.columns.str.strip().str.lower().str.replace(" ", "_", regex=False)

# 🔠 Ajuste opcional para facilitar filtros de texto em escolas
if "no_municipio" in df_escolas.columns:
    df_escolas["no_municipio"] = df_escolas["no_municipio"].astype(str).str.strip().str.upper()

print("✅ Pokémon:", df_poke.shape, " | colunas:", len(df_poke.columns))
print("✅ Escolas:", df_escolas.shape, " | colunas:", len(df_escolas.columns))

# 👀 Conferir primeiras colunas
list(df_poke.columns)[:15]



## 1) O que é `groupby()`?
`groupby()` agrupa linhas que têm o **mesmo valor** em uma (ou mais) colunas e permite calcular **estatísticas por grupo**.


In [ ]:

# 🎯 Exemplo básico — médias por tipo (Pokémon)
media_por_tipo = (
    df_poke.groupby('type_1')[['attack','defense','speed','hp']]
           .mean()
           .round(2)
)
media_por_tipo.head(10)



## 2) Funções de agregação comuns
- `count()`, `mean()`, `sum()`, `min()`, `max()`, `median()`, `std()`


In [ ]:

# 🎯 Agregações múltiplas com .agg()
agr = (
    df_poke.groupby('type_1')
           .agg(
               qtd=('name', 'count'),
               media_atk=('attack', 'mean'),
               media_def=('defense', 'mean'),
               media_spd=('speed', 'mean'),
               media_hp=('hp', 'mean'),
               atk_max=('attack', 'max'),
               spd_max=('speed', 'max')
           )
           .round(2)
           .sort_values('media_atk', ascending=False)
)
agr.head(10)



## 3) Agrupar por **múltiplas colunas**


In [ ]:

# 🎯 Média de Attack por combinação (type_1, generation) — se existir a coluna
col_gen = 'generation' if 'generation' in df_poke.columns else None

if col_gen:
    media_tipo_gen = df_poke.groupby(['type_1', col_gen])['attack'].mean().round(2)
    media_tipo_gen.head(10)
else:
    print("⚠️ Coluna 'generation' não encontrada; pulando exemplo 3.")



## 4) Selecionar **Top-N por grupo**
Duas estratégias úteis:  
- **Ordenar + `groupby().head(N)`**  
- `groupby().apply(lambda g: g.nlargest(N, 'coluna'))`


In [ ]:

# 🎯 Top 5 por ataque em cada tipo
top5_por_tipo = (
    df_poke
      .sort_values(['type_1','attack'], ascending=[True, False])
      .groupby('type_1')
      .head(5)[['name','type_1','attack','defense','speed']]
)
top5_por_tipo.head(15)


In [ ]:

# 🎯 Top 3 por velocidade em cada tipo (nlargest)
top3_speed_tipo = (
    df_poke.groupby('type_1', group_keys=False)
           .apply(lambda g: g.nlargest(3, 'speed'))[['name','type_1','speed','attack']]
)
top3_speed_tipo.head(12)



## 5) Formatação e renomeação para leitura


In [ ]:

tabela = (
    df_poke.groupby('type_1')
           .agg(qtd=('name','count'),
                atk_med=('attack','mean'),
                def_med=('defense','mean'),
                spd_med=('speed','mean'),
                hp_med=('hp','mean'))
           .round(1)
           .sort_values(['atk_med','spd_med'], ascending=[False, False])
)
tabela.reset_index().head(10)



## 6) (Opcional) Gráficos a partir de agregações


In [ ]:

import matplotlib.pyplot as plt

# 🎯 Barras: quantidade de Pokémon por tipo (Top 10)
contagem_tipo = df_poke['type_1'].value_counts().head(10)
ax = contagem_tipo.plot(kind='bar', figsize=(8,4))
ax.set_title('Top 10 tipos por quantidade de Pokémon')
ax.set_xlabel('type_1')
ax.set_ylabel('Quantidade')
plt.tight_layout()
plt.show()



## 7) Aplicando `groupby()` no dataset de Escolas


In [ ]:

# 🎯 7.1 Contagem de escolas por município (Top 10)
if 'no_municipio' in df_escolas.columns:
    top10_mun = df_escolas['no_municipio'].value_counts().head(10)
    top10_mun
else:
    print("⚠️ Coluna 'no_municipio' não encontrada.")

# 🎯 7.2 Agrupar por dependência administrativa (se existir)
if 'tp_dependencia' in df_escolas.columns and 'no_entidade' in df_escolas.columns:
    dep = df_escolas.groupby('tp_dependencia')['no_entidade'].count().sort_values(ascending=False)
    dep
else:
    print("⚠️ Colunas para dependência administrativa não encontradas.")



## ✅ Mini-Lab (entregar no Classroom)
**Arquivo:** `UC22_Aula07_Pratica_SeuNome.ipynb`

### Parte 1 — Pokémon
1. **Médias por tipo:** Tabela com `qtd`, `atk_med`, `def_med`, `spd_med`, `hp_med` por `type_1` (2 casas) e **ordene** por `atk_med` (desc) e `spd_med` (desc).  
2. **Top 5 ataque por tipo:** Os **5 maiores `attack`** dentro de cada `type_1` (mostre `name`, `type_1`, `attack`, `speed`).  
3. **Top 3 velocidade por tipo:** Usando `nlargest`, retorne os **3 mais rápidos** por `type_1`.  
4. **Médias por (type_1, generation)** – se existir `generation`: média de `attack` e `speed` por `(type_1, generation)`.

### Parte 2 — Escolas
1. **Top 10 municípios:** Mostre os 10 municípios com **maior número de escolas**.  
2. **Dependência administrativa:** Agrupe por `tp_dependencia` e mostre a contagem de escolas (ordem desc).

> **Dica:** Use `.reset_index()` quando precisar transformar o índice em coluna para ordenar/formatar.



### 📝 Espaço para suas respostas (execute os blocos e comente seus achados)


In [ ]:

# 🔧 Resposta Pokémon 1 — Médias por tipo (ordenadas)
resp1 = (
    df_poke.groupby('type_1')
           .agg(qtd=('name','count'),
                atk_med=('attack','mean'),
                def_med=('defense','mean'),
                spd_med=('speed','mean'),
                hp_med=('hp','mean'))
           .round(2)
           .sort_values(['atk_med','spd_med'], ascending=[False, False])
).reset_index()
resp1.head(15)


In [ ]:

# 🔧 Resposta Pokémon 2 — Top 5 ataque por tipo
resp2 = (
    df_poke.sort_values(['type_1','attack'], ascending=[True, False])
           .groupby('type_1')
           .head(5)[['name','type_1','attack','speed']]
)
resp2.head(20)


In [ ]:

# 🔧 Resposta Pokémon 3 — Top 3 velocidade por tipo (nlargest)
resp3 = (
    df_poke.groupby('type_1', group_keys=False)
           .apply(lambda g: g.nlargest(3, 'speed'))[['name','type_1','speed','attack']]
)
resp3.head(20)


In [ ]:

# 🔧 Resposta Pokémon 4 — Médias por (type_1, generation) se existir
if 'generation' in df_poke.columns:
    resp4 = (
        df_poke.groupby(['type_1','generation'])[['attack','speed']]
               .mean()
               .round(2)
               .reset_index()
    )
    resp4.head(20)
else:
    print("⚠️ Coluna 'generation' não encontrada.")


In [ ]:

# 🔧 Resposta Escolas 1 — Top 10 municípios
if 'no_municipio' in df_escolas.columns:
    resp_escolas1 = df_escolas['no_municipio'].value_counts().head(10)
    resp_escolas1
else:
    print("⚠️ Coluna 'no_municipio' não encontrada.")


In [ ]:

# 🔧 Resposta Escolas 2 — Dependência administrativa
if {'tp_dependencia','no_entidade'}.issubset(df_escolas.columns):
    resp_escolas2 = df_escolas.groupby('tp_dependencia')['no_entidade'].count().sort_values(ascending=False)
    resp_escolas2
else:
    print("⚠️ Colunas para dependência administrativa não encontradas.")



## 🧯 Erros Comuns e Dicas
- **Nome de coluna incorreto** → confira `df.columns`.  
- **Agregações em colunas de texto** não fazem sentido; selecione colunas numéricas.  
- **Ordenar antes de `groupby().head(N)`** é essencial para Top-N por grupo.  
- **`nlargest`** exige colunas numéricas e não aceita `object`.  
- **MultiIndex confuso** → aplique `.reset_index()` quando necessário.



## 📎 Conclusão
Você aprendeu a comparar grupos e extrair **insights** com `groupby()`: médias por tipo, top-N por grupo e contagens por categoria.  
Na **Aula 8**, vamos aprofundar **ordenação (`sort_values`) e rankings** para criar análises comparativas ainda mais ricas.



---
*Notebook gerado em: 2025-09-12 13:10:03*
